In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master('local[*]').appName('test').getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/03/05 17:15:25 WARN Utils: Your hostname, dashel resolves to a loopback address: 127.0.1.1; using 172.31.179.36 instead (on interface eth0)
24/03/05 17:15:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/05 17:15:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green = spark.read.parquet('data/green-taxi-data/parquet/*')

In [4]:
df_green.show()

+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|    pickup_datetime|   dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2|2020-01-23 17:50:43|2020-01-23 18:10:24|                 N|         1|          82|         258|              1|         3.85|       16.0|  1.0|    0.5|       0.0

In [5]:
df_green.registerTempTable('green')

/home/dashelruiz/anaconda3/envs/de_env/lib/python3.12/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [6]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', pickup_datetime) AS hour,
    PULocationID AS zone,    
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
GROUP BY
    1, 2
ORDER BY 
    1, 2
"""
)

In [7]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2008-12-31 22:00:00| 193|               0.0|             1|
|2008-12-31 23:00:00| 166|              11.3|             1|
|2008-12-31 23:00:00| 193|               0.0|             2|
|2008-12-31 23:00:00| 264|               0.0|             1|
|2009-01-01 00:00:00|   7|               9.8|             1|
|2009-01-01 00:00:00|  41|15.850000000000001|             2|
|2009-01-01 00:00:00|  42|               8.3|             1|
|2009-01-01 00:00:00|  74|               9.8|             1|
|2009-01-01 00:00:00|  75| 98.39999999999999|             2|
|2009-01-01 00:00:00| 116|               7.3|             1|
|2009-01-01 00:00:00| 130|              11.8|             1|
|2009-01-01 00:00:00| 193|             12.35|             8|
|2009-01-01 00:00:00| 264|               0.0|             1|
|2009-01-01 01:00:00|  4

In [16]:
df_green_revenue.repartition(20).write.parquet('data/report/revenue/green', mode='overwrite')

24/03/05 17:36:25 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/05 17:36:25 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/03/05 17:36:25 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/03/05 17:36:25 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/03/05 17:36:25 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
24/03/05 17:36:25 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
24/03/05 17:36:25 WARN MemoryManager: Total allocation exceeds 95.

In [8]:
df_yellow = spark.read.parquet('data/yellow-taxi-data/parquet/*')

In [9]:
df_yellow.registerTempTable('yellow')

In [10]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', pickup_datetime) AS hour,
    PULocationID AS zone,    
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
GROUP BY
    1, 2
ORDER BY 
    1, 2
"""
)

In [11]:
df_yellow_revenue.show()

24/03/05 17:16:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/03/05 17:16:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/03/05 17:16:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/03/05 17:16:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+-------------------+----+------+--------------+
|               hour|zone|amount|number_records|
+-------------------+----+------+--------------+
|2002-12-31 23:00:00| 193|   0.0|             1|
|2002-12-31 23:00:00| 264|   0.0|             1|
|2003-01-01 00:00:00| 193|   0.0|             1|
|2003-01-05 07:00:00| 264|   0.0|             1|
|2003-03-26 21:00:00| 193|  3.31|             1|
|2004-04-04 04:00:00| 162|  12.3|             1|
|2008-12-31 22:00:00| 113|  10.8|             1|
|2008-12-31 22:00:00| 164|  27.3|             1|
|2008-12-31 22:00:00| 238|  26.3|             1|
|2008-12-31 23:00:00|  25|  16.8|             1|
|2008-12-31 23:00:00|  43|  53.5|             2|
|2008-12-31 23:00:00|  75|  11.3|             1|
|2008-12-31 23:00:00|  79|  23.8|             1|
|2008-12-31 23:00:00|  85|   5.3|             1|
|2008-12-31 23:00:00|  90|   9.8|             1|
|2008-12-31 23:00:00| 132|302.12|             7|
|2008-12-31 23:00:00| 141|  23.6|             2|
|2008-12-31 23:00:00

In [17]:
df_yellow_revenue.repartition(20).write.parquet('data/report/revenue/yellow', mode='overwrite')

24/03/05 17:36:42 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/03/05 17:36:42 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/03/05 17:36:42 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/03/05 17:36:42 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/03/05 17:36:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/05 17:36:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/03/05 17:36:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/03/05 17:36:45 WARN MemoryManager: Total allocation exceeds 95.

In [12]:
df_green_revenue_tem = df_green_revenue \
        .withColumnRenamed('amount', 'green_amount') \
        .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tem = df_yellow_revenue \
        .withColumnRenamed('amount', 'yellow_amount') \
        .withColumnRenamed('number_records', 'yellow_number_records')

In [13]:
df_join = df_green_revenue_tem.join(df_yellow_revenue_tem, on=['hour', 'zone'], how='outer')

In [ ]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [15]:
df_join.show()

24/03/05 17:32:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/03/05 17:32:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/03/05 17:32:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/03/05 17:32:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/03/05 17:32:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/03/05 17:32:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/03/05 17:32:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/03/05 17:32:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2003-01-05 07:00:00| 264|              NULL|                NULL|               0.0|                    1|
|2008-12-31 23:00:00| 162|              NULL|                NULL|              26.6|                    2|
|2008-12-31 23:00:00| 237|              NULL|                NULL|              35.2|                    4|
|2009-01-01 00:00:00|  24|              NULL|                NULL|              11.0|                    1|
|2009-01-01 00:00:00|  41|15.850000000000001|                   2|              11.8|                    1|
|2009-01-01 00:00:00|  79|              NULL|                NULL|35.760000000000005|                    3|
|2009-01-01 00:00:00| 229|  

In [19]:
df_join = spark.read.parquet('data/report/revenue/total')

In [20]:
df_zones = spark.read.parquet('zones')

In [21]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [22]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [23]:
df_result.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|  Borough|                Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|2008-12-31 23:00:00| 193|               0.0|                   2|               3.3|                    7|       193|   Queens|Queensbridge/Rave...|   Boro Zone|
|2008-12-31 23:00:00| 231|              NULL|                NULL|             32.76|                    1|       231|Manhattan|TriBeCa/Civic Center| Yellow Zone|
|2008-12-31 23:00:00| 233|              NULL|                NULL|               8.3|                    1|       233|Manhattan| UN/Turtle Bay South| Yellow Zone|
|2008-12-31 23:00:00| 

In [ ]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')